In [1]:
import pandas as pd


### 第六章 连接
#### 关系型连接
将数据集连接起来，这个是一个在实际的数据处理中经常用到的一个操作，简单的一个series连接到一个dataframe数据上，可以直接赋予一个列名，然后赋值就可以。
在pandas中有连接函数merge和join，它分为左连接、右连接、内连接、外连接


#### 值连接


In [2]:
df1 = pd.DataFrame({'Name':['San Zhang','Si Li'],'Age':[20,30]})
df2 = pd.DataFrame({'Name':['Si Li','Wu Wang'],'Gender':['F','M']})
df1.merge(df2, on='Name', how='left')    
# 在name上实现左连接，以左边的键为准

,Name,Age,Gender
0,San Zhang,20,NaN
1,Si Li,30,F


In [4]:
# 如果想要连接的列不具备相同的列名，可以通过left_on和right_on指定
df1 = pd.DataFrame({'df1_name':['San Zhang','Si Li'],'Age':[20,30]})
df2 = pd.DataFrame({'df2_name':['Si Li','Wu Wang'],'Gender':['F','M']})
df1.merge(df2, left_on='df1_name', right_on='df2_name', how='left')

,df1_name,Age,df2_name,Gender
0,San Zhang,20,NaN,NaN
1,Si Li,30,Si Li,F


In [5]:
# 如果两个表中的列出现了重复的列名，那么可以通过 suffixes 参数指定
df1 = pd.DataFrame({'Name':['San Zhang'],'Grade':[70]})
df2 = pd.DataFrame({'Name':['San Zhang'],'Grade':[80]})
df1.merge(df2, on='Name', how='left', suffixes=['_Chinese','_Math'])

,Name,Grade_Chinese,Grade_Math
0,San Zhang,70,80


In [8]:
# 在某些时候出现重复元素是麻烦的，例如两位同学来自不同的班级，但是姓名相同，
# 这种时候就要指定 on 参数为多个列使得正确连接
df1 = pd.DataFrame({'Name':['San Zhang', 'San Zhang'],'Age':[20, 21],
                    'Class':['one', 'two']})
df2 = pd.DataFrame({'Name':['San Zhang', 'San Zhang'],'Gender':['F', 'M'],
                    'Class':['two', 'one']})
df1

,Name,Age,Class
0,San Zhang,20,one
1,San Zhang,21,two


In [9]:
df2

,Name,Gender,Class
0,San Zhang,F,two
1,San Zhang,M,one


In [10]:
df1.merge(df2, on='Name', how='left')

,Name,Age,Class_x,Gender,Class_y
0,San Zhang,20,one,F,two
1,San Zhang,20,one,M,one
2,San Zhang,21,two,F,two
3,San Zhang,21,two,M,one


In [11]:
df1.merge(df2, on=['Name', 'Class'], how='left')

,Name,Age,Class,Gender
0,San Zhang,20,one,M
1,San Zhang,21,two,F


#### 索引连接
所谓索引连接，就是把索引当作键，因此这和值连接本质上没有区别， pandas 中利用 join 函数来处理索引连接，它的参数选择要少于 merge ，除了必须的 on 和 how 之外，可以对重复的列指定左右后缀 lsuffix 和 rsuffix 。其中， on 参数指索引名，单层索引时省略参数表示按照当前索引连接

In [13]:
df1 = pd.DataFrame({'Age':[20,30]},index=pd.Series(['San Zhang','Si Li'],name='Name'))
df2 = pd.DataFrame({'Gender':['F','M']},index=pd.Series(['Si Li','Wu Wang'],name='Name'))
df1.join(df2, how='left')

,Age,Gender
Name,,
San Zhang,20,NaN
Si Li,30,F


In [14]:
df1 = pd.DataFrame({'Grade':[70]}, index=pd.Series(['San Zhang'],name='Name'))
df2 = pd.DataFrame({'Grade':[80]},index=pd.Series(['San Zhang'],name='Name'))
df1.join(df2, how='left', lsuffix='_Chinese', rsuffix='_Math')

,Grade_Chinese,Grade_Math
Name,,
San Zhang,70,80


In [15]:
df1 = pd.DataFrame({'Age':[20,21]},
                   index=pd.MultiIndex.from_arrays(
                       [['San Zhang', 'San Zhang'],['one', 'two']],
                       names=('Name','Class')))

In [16]:
df2 = pd.DataFrame({'Gender':['F', 'M']},
                   index=pd.MultiIndex.from_arrays(
                       [['San Zhang', 'San Zhang'],['two', 'one']],
                       names=('Name','Class')))

In [17]:
df1

Age
Name      Class     
San Zhang one     20
          two     21

In [18]:
df2

Gender
Name      Class       
San Zhang two        F
          one        M

In [19]:
df1.join(df2)

Age Gender
Name      Class            
San Zhang one     20      M
          two     21      F

### 方向连接
- concat 把两个表或者多个表按照纵向或者横向拼接

在 concat 中，最常用的有三个参数，它们是 axis, join, keys ，分别表示拼接方向，连接形式，以及在新表中指示来自于哪一张旧表的名字。这里需要特别注意， join 和 keys 与之前提到的 join 函数和键的概念没有任何关系。

在默认状态下的 axis=0 ，表示纵向拼接多个表，常常用于多个样本的拼接；而 axis=1 表示横向拼接多个表，常用于多个字段或特征的拼接

In [20]:
df1 = pd.DataFrame({'Name':['San Zhang','Si Li'],
                    'Age':[20,30]})
df2 = pd.DataFrame({'Name':['Wu Wang'], 'Age':[40]})
pd.concat([df1, df2])

,Name,Age
0,San Zhang,20
1,Si Li,30
0,Wu Wang,40


In [21]:
df2 = pd.DataFrame({'Grade':[80, 90]})
df3 = pd.DataFrame({'Gender':['M', 'F']})
pd.concat([df1, df2, df3], 1)

,Name,Age,Grade,Gender
0,San Zhang,20,80,M
1,Si Li,30,90,F


虽然说 concat 不是处理关系型合并的函数，但是它仍然是关于索引进行连接的。纵向拼接会根据列索引对其，默认状态下 join=outer ，表示保留所有的列，并将不存在的值设为缺失； join=inner ，表示保留两个表都出现过的列。横向拼接则根据行索引对齐， join 参数可以类似设置

In [22]:
df2 = pd.DataFrame({'Name':['Wu Wang'], 'Gender':['M']})

In [23]:
pd.concat([df1, df2])

,Name,Age,Gender
0,San Zhang,20.0,NaN
1,Si Li,30.0,NaN
0,Wu Wang,NaN,M


In [24]:
df2 = pd.DataFrame({'Grade':[80, 90]}, index=[1, 2])

In [25]:
pd.concat([df1, df2], 1)

,Name,Age,Grade
0,San Zhang,20.0,NaN
1,Si Li,30.0,80.0
2,NaN,NaN,90.0


In [26]:
pd.concat([df1, df2], axis=1, join='inner')

,Name,Age,Grade
1,Si Li,30,80


当确认要使用多表直接的方向合并时，尤其是横向的合并，可以先用 reset_index 方法恢复默认整数索引再进行合并，防止出现由索引的误对齐和重复索引的笛卡尔积带来的错误结果

In [27]:
df1 = pd.DataFrame({'Name':['San Zhang','Si Li'],'Age':[20,21]})
df2 = pd.DataFrame({'Name':['Wu Wang'],'Age':[21]})
pd.concat([df1, df2], keys=['one', 'two'])

Name  Age
one 0  San Zhang   20
    1      Si Li   21
two 0    Wu Wang   21

#### 序列与表的合并
利用 concat 可以实现多个表之间的方向拼接，如果想要把一个序列追加到表的行末或者列末，则可以分别使用 append 和 assign 方法。

在 append 中，如果原表是默认整数序列的索引，那么可以使用 ignore_index=True 对新序列对应索引的自动标号，否则必须对 Series 指定 name 属性

In [28]:
s = pd.Series(['Wu Wang', 21], index = df1.columns)
df1.append(s, ignore_index=True)

,Name,Age
0,San Zhang,20
1,Si Li,21
2,Wu Wang,21


对于 assign 而言，虽然可以利用其添加新的列，但一般通过 df['new_col'] = ... 的形式就可以等价地添加新列。同时，使用 [] 修改的缺点是它会直接在原表上进行改动，而 assign 返回的是一个临时副本

In [29]:
s = pd.Series([80, 90])
df1.assign(Grade=s)

,Name,Age,Grade
0,San Zhang,20,80
1,Si Li,21,90


In [30]:
df1['Grade'] = s
df1

,Name,Age,Grade
0,San Zhang,20,80
1,Si Li,21,90


### 类连接操作
#### 比较 compare 它能够比较两个表或者序列的不同处并将其汇总展示


In [31]:
df1 = pd.DataFrame({'Name':['San Zhang', 'Si Li', 'Wu Wang'],
                    'Age':[20, 21 ,21],
                    'Class':['one', 'two', 'three']})

In [32]:
df2 = pd.DataFrame({'Name':['San Zhang', 'Li Si', 'Wu Wang'],
                    'Age':[20, 21 ,21],
                    'Class':['one', 'two', 'Three']})

In [33]:
df1.compare(df2)

Name         Class       
    self  other   self  other
1  Si Li  Li Si    NaN    NaN
2    NaN    NaN  three  Three

In [34]:
df1.compare(df2, keep_shape=True)

Name         Age        Class       
    self  other self other   self  other
0    NaN    NaN  NaN   NaN    NaN    NaN
1  Si Li  Li Si  NaN   NaN    NaN    NaN
2    NaN    NaN  NaN   NaN  three  Three

#### 组合
combine 函数能够让两张表按照一定的规则进行组合，在进行规则比较时会自动进行列索引的对齐。对于传入的函数而言，每一次操作中输入的参数是来自两个表的同名 Series ，依次传入的列是两个表列名的并集，例如下面这个例子会依次传入 A,B,C,D 四组序列，每组为左右表的两个序列。同时，进行 A 列比较的时候， s1 指代的就是一个全空的序列，因为它在被调用的表中并不存在，并且来自第一个表的序列索引会被 reindex 成两个索引的并集。具体的过程可以通过在传入的函数中插入适当的 print 方法查看

In [35]:
def choose_min(s1, s2):
    s2 = s2.reindex_like(s1)
    res = s1.where(s1<s2, s2)
    res = res.mask(s1.isna()) # isna表示是否为缺失值，返回布尔序列
    return res

In [36]:
df1 = pd.DataFrame({'A':[1,2], 'B':[3,4], 'C':[5,6]})

In [37]:
df2 = pd.DataFrame({'B':[5,6], 'C':[7,8], 'D':[9,10]}, index=[1,2])

In [38]:
df1.combine(df2, choose_min)

,A,B,C,D
0,NaN,NaN,NaN,NaN
1,NaN,4.0,6.0,NaN
2,NaN,NaN,NaN,NaN


In [39]:
df1.combine(df2, choose_min, overwrite=False)

,A,B,C,D
0,1.0,NaN,NaN,NaN
1,2.0,4.0,6.0,NaN
2,NaN,NaN,NaN,NaN


In [41]:
import numpy as np
df1 = pd.DataFrame({'A':[1,2], 'B':[3,np.nan]})
df2 = pd.DataFrame({'A':[5,6], 'B':[7,8]}, index=[1,2])
df1.combine_first(df2)

,A,B
0,1.0,3.0
1,2.0,7.0
2,6.0,8.0
